In [1]:
import pyspark
import pyspark.sql.functions as f

#1. Read the CSV file provided into a Spark DataFrame.

In [2]:


from pyspark.sql import SparkSession
import pyspark.sql.functions as f

# Create SparkSession
spark = (SparkSession.builder 
    .appName("TitanicAnalysis") 
    .master("spark://spark:7077")  
    .getOrCreate())




In [4]:
# Read CSV (use container path if inside Docker)
df = spark.read.csv("/data/titanic.csv", header=True, inferSchema=True)

df.show(5)
df.printSchema()

+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|PassengerId|Survived|Pclass|                Name|gender| Age|SibSp|Parch|          Ticket|   Fare|Cabin|Embarked|
+-----------+--------+------+--------------------+------+----+-----+-----+----------------+-------+-----+--------+
|          1|       0|     3|Braund, Mr. Owen ...|  male|22.0|    1|    0|       A/5 21171|   7.25| NULL|       S|
|          2|       1|     1|Cumings, Mrs. Joh...|female|38.0|    1|    0|        PC 17599|71.2833|  C85|       C|
|          3|       1|     3|Heikkinen, Miss. ...|female|26.0|    0|    0|STON/O2. 3101282|  7.925| NULL|       S|
|          4|       1|     1|Futrelle, Mrs. Ja...|female|35.0|    1|    0|          113803|   53.1| C123|       S|
|          5|       0|     3|Allen, Mr. Willia...|  male|35.0|    0|    0|          373450|   8.05| NULL|       S|
+-----------+--------+------+--------------------+------+----+-----+-----+------

2. Count the number of males and females who had their Cabin as null.

In [5]:
df.filter(f.col("Cabin").isNull()).groupby("gender").count().show()

+------+-----+
|gender|count|
+------+-----+
|female|  217|
|  male|  470|
+------+-----+



3. Find the average ages of passengers.

In [6]:
df.groupBy("gender").agg(f.avg("Age").alias("Average Age")).show()

+------+------------------+
|gender|       Average Age|
+------+------------------+
|female|27.915708812260537|
|  male| 30.72664459161148|
+------+------------------+



In [7]:
average_age =df.select(f.avg("Age").alias("Average Age"))
average_age.show()


+-----------------+
|      Average Age|
+-----------------+
|29.69911764705882|
+-----------------+



4. Fill in the missing age values with this average value.

In [8]:
df.filter(f.col("Age").isNull()).count()

177

In [9]:
mean_age = df.select(f.avg("Age")).collect()[0][0]
print("Mean Age:", mean_age)

df_filled = df.fillna({"Age": mean_age})

Mean Age: 29.69911764705882


In [10]:
df_filled.filter(f.col("Age").isNull()).count()

0

5. Save the output to a CSV file in HDFS (depi_folder).

In [16]:
df_filled.write.mode("overwrite").csv("hdfs://namenode:9000/depi_folder", header=True)


6. Count the total number of passengers who survived and those who did not.

In [17]:
survived =df_filled.filter(f.col("Survived")== 1).count()
death = df_filled.filter(f.col("Survived")== 0).count()
print(f"number of passengers survived is  {survived} \n and number of deaths is {death}")

number of passengers survived is  342 
 and number of deaths is 549


7. Find the top 5 most common embarkation ports among passengers.

In [18]:
df.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [19]:
df.select("Embarked").distinct().show()

+--------+
|Embarked|
+--------+
|       Q|
|       C|
|       S|
|    NULL|
+--------+



In [20]:
df.groupby("Embarked").count().orderBy(f.desc("count")).show()

+--------+-----+
|Embarked|count|
+--------+-----+
|       S|  644|
|       C|  168|
|       Q|   77|
|    NULL|    2|
+--------+-----+



In [22]:
import pandas as pd
data = pd.read_csv("/data/titanic.csv")
data.head(5)


,PassengerId,Survived,Pclass,Name,gender,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


8. Calculate the survival rate by passenger class (Pclass).

In [23]:
survival_stats = (
    df.groupBy("Pclass")
      .agg(
          f.count("*").alias("Total"),
          f.sum(f.col("Survived")).alias("Survived_Count")
      )
      .withColumn("Survival_Rate", f.col("Survived_Count") / f.col("Total"))
      .orderBy("Pclass")
)

survival_stats.show()


+------+-----+--------------+-------------------+
|Pclass|Total|Survived_Count|      Survival_Rate|
+------+-----+--------------+-------------------+
|     1|  216|           136| 0.6296296296296297|
|     2|  184|            87|0.47282608695652173|
|     3|  491|           119|0.24236252545824846|
+------+-----+--------------+-------------------+



9. Determine the maximum, minimum, and average fare paid by passengers.

In [24]:
df.agg(f.max("Fare")).show()
df.agg(f.min("Fare")).show()
df.agg(f.mean("Fare")).show()

+---------+
|max(Fare)|
+---------+
| 512.3292|
+---------+

+---------+
|min(Fare)|
+---------+
|      0.0|
+---------+

+----------------+
|       avg(Fare)|
+----------------+
|32.2042079685746|
+----------------+



10. Write a Spark job to count the number of passengers in each age group (e.g., 0–18, 19–35, 36–60, 60+).

In [25]:
df_age_grouped = df_filled.withColumn(
    "AgeGroup",
    f.when(f.col("Age") <= 18, "0-18")
     .when((f.col("Age") > 18) & (f.col("Age") <= 35), "19-35")
     .when((f.col("Age") > 35) & (f.col("Age") <= 60), "36-60")
     .otherwise("60+")
)

df_age_grouped.groupby("AgeGroup").count().show()

+--------+-----+
|AgeGroup|count|
+--------+-----+
|   19-35|  535|
|     60+|   22|
|   36-60|  195|
|    0-18|  139|
+--------+-----+



11. Create a new directory in HDFS called titanic_lab and list its contents.

12. Upload the Titanic dataset from your local machine to the titanic_lab directory in HDFS.

13. Use chmod command to change the permissions of the Titanic dataset file to 777 (full permissionsfor all)

14. Use-cat to display the first 20 lines of the Titanic dataset stored in HDFS.

15- Move the processed output file from titanic_lab to a new directory in HDFS called titanic_results using hdfs dfs -mv.


In [ ]:
#all commands
!root@c9e0bf9264de:/# hdfs dfs -mkdir  /titanic_lab
!root@c9e0bf9264de:/# hdfs dfs -ls /titanic_lab


!root@c9e0bf9264de:/# hdfs dfs -chmod 777 /titanic_lab/titanic.csv
!root@c9e0bf9264de:/# hdfs dfs -cat /titanic_lab/titanic.csv | head -n 20 

#1,0,3,"Braund, Mr. Owen Harris",male,22,1,0,A/5 21171,7.25,,S
# 2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Thayer)",female,38,1,0,PC 17599,71.2833,C85,C
# 3,1,3,"Heikkinen, Miss. Laina",female,26,0,0,STON/O2. 3101282,7.925,,S
# 4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35,1,0,113803,53.1,C123,S
# 5,0,3,"Allen, Mr. William Henry",male,35,0,0,373450,8.05,,S
# 6,0,3,"Moran, Mr. James",male,,0,0,330877,8.4583,,Q
# 7,0,1,"McCarthy, Mr. Timothy J",male,54,0,0,17463,51.8625,E46,S



!root@c9e0bf9264de:/# hdfs dfs -mkdir /titanic_results
!hdfs dfs -mv /titanic_lab/titanic.csv /titanic_results/
